In [1]:
from distiller.utils import read_jsonl, write_jsonl

In [2]:
c2e = read_jsonl("./data/snli/examples/contradiction_entailment.jsonl")
e2c = read_jsonl("./data/snli/examples/entailment_contradiction.jsonl")
c2n = read_jsonl("./data/snli/examples/contradiction_neutral.jsonl")
e2n = read_jsonl("./data/snli/examples/entailment_neutral.jsonl")
n2c = read_jsonl("./data/snli/examples/neutral_contradiction.jsonl")

In [3]:
from hydra import compose, initialize

def load_template(task, template_name):
    initialize(config_path="templates", version_base="1.3")
    cfg = compose(config_name=task)
    template = cfg.templates[template_name]
    return template

template = load_template("snli", "masked_cad_premise")

In [5]:
prompt_template = template.template
prompt_template

"statement: {{sentence1.replace(span, '[blank]')}} \nconclusion: {{sentence2}} \n[blank] should be"

In [6]:
from jinja2 import BaseLoader, Environment, meta

env = Environment(loader=BaseLoader)

def compose_prompt(template, render_items):
    prompt = env.from_string(template).render(**render_items)
    return prompt

In [10]:
item = c2e[0]
item['sentence1'] = item['premise']
item['sentence2'] = item['hypothesis']
item['span'] = item['span_changed']
item

{'premise': 'a young mother carrying her sleeping daughter.',
 'hypothesis': 'the woman is asleep',
 'label': 'contradiction',
 'new_label': 'entailment',
 'span_changed': 'carrying her sleeping daughter',
 'span_to': 'is having a good night dream on the couch',
 'sentence1': 'a young mother carrying her sleeping daughter.',
 'sentence2': 'the woman is asleep',
 'span': 'carrying her sleeping daughter'}

In [9]:
compose_prompt(prompt_template, item)

'statement: a young mother [blank]. \nconclusion: the woman is asleep \n[blank] should be'

In [11]:
import uuid

c2e_convert = []
for item in c2e:
    item['sentence1'] = item['premise']
    item['sentence2'] = item['hypothesis']
    item['span'] = item['span_changed']
    prompt = compose_prompt(prompt_template, item)
    output = item["span_to"]

    guid = str(uuid.uuid4())
    example = {
        "guid": guid,
        "input": prompt,
        "output": output,
    }
    c2e_convert.append(example)

In [13]:
e2c_convert = []
for item in e2c:
    item['sentence1'] = item['premise']
    item['sentence2'] = item['hypothesis']
    item['span'] = item['span_changed']
    prompt = compose_prompt(prompt_template, item)
    output = item["span_to"]

    guid = str(uuid.uuid4())
    example = {
        "guid": guid,
        "input": prompt,
        "output": output,
    }
    e2c_convert.append(example)

In [14]:
c2n_convert = []
for item in c2n:
    item['sentence1'] = item['premise']
    item['sentence2'] = item['hypothesis']
    item['span'] = item['span_changed']
    prompt = compose_prompt(prompt_template, item)
    output = item["span_to"]

    guid = str(uuid.uuid4())
    example = {
        "guid": guid,
        "input": prompt,
        "output": output,
    }
    c2n_convert.append(example)

In [15]:
e2n_convert = []
for item in e2n:
    item['sentence1'] = item['premise']
    item['sentence2'] = item['hypothesis']
    item['span'] = item['span_changed']
    prompt = compose_prompt(prompt_template, item)
    output = item["span_to"]

    guid = str(uuid.uuid4())
    example = {
        "guid": guid,
        "input": prompt,
        "output": output,
    }
    e2n_convert.append(example)

In [16]:
n2c_convert = []
for item in n2c:
    item['sentence1'] = item['premise']
    item['sentence2'] = item['hypothesis']
    item['span'] = item['span_changed']
    prompt = compose_prompt(prompt_template, item)
    output = item["span_to"]

    guid = str(uuid.uuid4())
    example = {
        "guid": guid,
        "input": prompt,
        "output": output,
    }
    n2c_convert.append(example)

In [17]:
example_dict = {
    "contradiction_entailment": c2e_convert,
    "entailment_contradiction": e2c_convert,
    "contradiction_neutral": c2n_convert,
    "entailment_neutral": e2n_convert,
    "neutral_contradiction": n2c_convert,
}

In [21]:
write_jsonl(n2c_convert, "./data/snli/examples/neutral_contradiction.jsonl")